# Notebook for M3 assesment

### Import of libraries

In [35]:
import grid2op
from Grid2OpPowsybl.src.Backend.PowsyblBackend import PowsyblBackend
import os
from pathlib import Path

In [36]:
backend = PowsyblBackend()

In [37]:
backend.load_grid('','Grid2OpPowsybl/src/data_test/test_case14.xiidm')

<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


In [43]:
backend.name_load

array(['B2-L', 'B3-L', 'B4-L', 'B9-L', 'B5-L', 'B6-L', 'B10-L', 'B11-L',
       'B12-L', 'B13-L', 'B14-L'], dtype='<U5')

In [38]:
type(backend).set_env_name("BaseTestLoadingCase")
backend.assert_grid_correct()

In [44]:
conv=backend.runpf()

In [45]:
print(conv)

True


In [46]:
backend.assert_grid_correct_after_powerflow()

In [49]:
backend.get_line_overflow()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])